# Section 1

In [41]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from bs4 import MarkupResemblesLocatorWarning
import warnings

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

df = pd.read_csv("./datasets/Assassin.csv")
df.head()


,sender,receiver,date,subject,body,label,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,"Thu, 22 Aug 2002 18:26:25 +0700",Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...","Thu, 22 Aug 2002 12:46:18 +0100",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,"Thu, 22 Aug 2002 13:52:38 +0100",[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,"Thu, 22 Aug 2002 09:15:25 -0400",[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,"Thu, 22 Aug 2002 14:38:22 +0100",Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


In [42]:
# Clean all column names by stripping leading/trailing whitespace
# df.columns = df.columns.str.strip()
df.head()

,sender,receiver,date,subject,body,label,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,"Thu, 22 Aug 2002 18:26:25 +0700",Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...","Thu, 22 Aug 2002 12:46:18 +0100",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,"Thu, 22 Aug 2002 13:52:38 +0100",[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,"Thu, 22 Aug 2002 09:15:25 -0400",[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,"Thu, 22 Aug 2002 14:38:22 +0100",Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


In [43]:
df = df.drop(columns=['sender', 'receiver','date','subject', 'urls'])

In [44]:
# df.head()
df.dtypes
df.isnull().sum()

body     1
label    0
dtype: int64

In [45]:
df = df.dropna()

In [46]:
df.isnull().sum()

body     0
label    0
dtype: int64

In [47]:
df.duplicated().sum()

np.int64(0)

In [48]:
def clean_email_body(html_text):
    # 1. Parse HTML and extract plain text
    try:
        soup = BeautifulSoup(html_text, 'html.parser')
        text = soup.get_text()
    except:
        # Handle cases where the body might not be HTML (e.g., just plain text)
        text = html_text

    # 2. Normalize whitespace 
    text = re.sub(r'\s+', ' ', text)

    # 3. Remove URLs, emails, and punctuation
    text = re.sub(r'http\S+', ' ', text)  # Replace URLs with a space
    text = re.sub(r'\S+@\S+', ' ', text) # Replace emails with a space
    text = re.sub(r'[^a-zA-Z\s]', ' ', text) # Replace non-letters with a space

    # 4. Final cleanup: lowercase and strip leading/trailing spaces
    text = text.lower().strip()
    
    return text

df["cleaned_email_body"] = df["body"].apply(clean_email_body)
df.head()


,body,label,cleaned_email_body
0,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,date wed aug from ch...
1,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,martin a posted tassos papadopoulos the gree...
2,Man Threatens Explosion In Moscow \n\nThursday...,0,man threatens explosion in moscow thursday aug...
3,Klez: The Virus That Won't Die\n \nAlready the...,0,klez the virus that won t die already the mos...
4,"> in adding cream to spaghetti carbonara, whi...",0,in adding cream to spaghetti carbonara which ...


In [49]:
df = df.drop(columns="body")

In [50]:
df.head()

,label,cleaned_email_body
0,0,date wed aug from ch...
1,0,martin a posted tassos papadopoulos the gree...
2,0,man threatens explosion in moscow thursday aug...
3,0,klez the virus that won t die already the mos...
4,0,in adding cream to spaghetti carbonara which ...


In [51]:

# The regex pattern of words you want to count
financial_urgency_pattern = r"invoice|payment|due|overdue|charge|fee|bank|account"

security_alerts_pattern = r"password|login|security|alert|unauthorized|suspicious|compromised"

scam_giveaway_language_pattern = r"congratulations|winner|prize|free|claim|reward|lotto|cash"

impersonation_generic_greetings_pattern = r"dear user|dear customer|valued customer|account holder"

threats_or_consequences_pattern = r"locked|suspended|action required|limited|restricted|unusual activity"

# Create the new column 'Keyword Count'
# df["Keyword Count"] = df["Email Text"].str.count(financial_urgency_pattern, flags=re.IGNORECASE)
# df.groupby('Email Type')['Keyword Count'].count()
unique_count_col1 = df['Email Text'].nunique()
print(f"Number of unique entries in 'col1': {unique_count_col1}")
count_col1 = df['Email Text'].count()
print(f"Number of entries in 'col1': {count_col1}")

KeyError: 'Email Text'

In [ ]:
df["Keyword2 Count"] = df["Email Text"].str.count(security_alerts_pattern, flags=re.IGNORECASE)
df.groupby('Email Type')['Keyword2 Count'].sum()

Email Type
Phishing Email    1542.0
Safe Email        1873.0
Name: Keyword2 Count, dtype: float64

In [ ]:
# new_df = df.sort_values(by="Keyword Count", ascending=False)
# pd.set_option('display.max_colwidth', 1)

In [ ]:
df["Keyword3 Count"] = df["Email Text"].str.count(financial_urgency_pattern, flags=re.IGNORECASE)
df.groupby('Email Type')['Keyword3 Count'].sum()
# another_df = df.sort_values(by="Keyword3 Count", ascending=False)
# another_df

Email Type
Phishing Email     6639.0
Safe Email        14732.0
Name: Keyword3 Count, dtype: float64